In [1]:
import os, sys, re, gc

import numpy as np
import pandas as pd
from IPython.display import display, HTML, clear_output

import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
datasets = []

for fn in glob.glob('dataset/df/*.parquet'):
    _df = pd.read_parquet(fn)
    _src_name = re.sub('^.*[/\\\]','', fn).split('.')[0]
    _df['src_name']=_src_name
    print(_src_name)
    datasets.append(_df)
    
df = pd.concat(datasets)
gc.collect()
   

angular
bootstrap-form
bootstrap-forms
bootstrap
complex-table
contact-form
dates
different-elements
html-5
login
metals-and-colors
mobile-and-html-5
performance
react-ant
search
simple-table
support
table-with-pages
user-table


4

In [3]:
df.columns

Index(['parent_id', 'element_id', 'tag_name', 'x', 'y', 'height', 'width',
       'displayed', 'enabled', 'selected', 'text', 'is_hover',
       'base64png_before_hover', 'base64png_after_hover', 'attr_class',
       'attr_onclick', 'attr_type', 'attr_role', 'attr_id', 'src_name'],
      dtype='object')

In [4]:
display(df[df.displayed].head())

,parent_id,element_id,tag_name,x,y,height,width,displayed,enabled,selected,text,is_hover,base64png_before_hover,base64png_after_hover,attr_class,attr_onclick,attr_type,attr_role,attr_id,src_name
0,None,91df6dab-a0b3-4a93-9851-fb1c6ac64640,html,0,0,44785.0,1183.0,True,True,False,None,True,None,None,,None,None,None,,angular
171,91df6dab-a0b3-4a93-9851-fb1c6ac64640,dee6f6bb-f389-4684-b1c0-da2f854c8515,body,0,0,44802.0,1200.0,True,True,False,,True,None,None,,None,None,None,,angular
172,dee6f6bb-f389-4684-b1c0-da2f854c8515,63813632-9941-47d9-8f7e-c198a4abeb25,header,0,0,60.0,1200.0,True,True,False,None,True,None,None,,None,None,None,,angular
173,63813632-9941-47d9-8f7e-c198a4abeb25,34eb03d4-5c52-4150-8807-8521b872f30b,div,0,0,60.0,1200.0,True,True,False,None,True,None,None,uui-header dark-gray,None,None,None,,angular
174,34eb03d4-5c52-4150-8807-8521b872f30b,62b82ba2-74df-4873-9c22-22e491d4a928,nav,0,0,0.0,1200.0,True,True,False,None,False,None,None,,None,None,navigation,,angular


In [5]:
display(HTML(f"<h2>Dataset shape: {df.shape}</h2>"))

In [6]:
pd.DataFrame(df.groupby('tag_name').size()).sort_values(by=0, ascending=False).head(50)

,0
tag_name,
div,4357
td,2486
a,1422
span,1416
li,1007
tr,653
option,578
label,536
input,535


In [7]:
df[df.displayed].groupby('tag_name').size()

tag_name
a                                                975
autocomplete-auto-active-first-option-example      1
autocomplete-display-example                       1
autocomplete-filter-example                        1
autocomplete-optgroup-example                      1
                                                ... 
tree-dynamic-example                               1
tree-flat-overview-example                         1
tree-loadmore-example                              1
tree-nested-overview-example                       1
ul                                               158
Length: 253, dtype: int64

In [8]:
df[~df.base64png_before_hover.isnull()].shape

(7326, 20)

In [9]:
df[df.is_hover].shape, df[df.enabled].shape, df[df.selected].shape

((4197, 20), (17026, 20), (73, 20))

In [10]:
df[df.selected]

,parent_id,element_id,tag_name,x,y,height,width,displayed,enabled,selected,text,is_hover,base64png_before_hover,base64png_after_hover,attr_class,attr_onclick,attr_type,attr_role,attr_id,src_name
466,4863170e-48cb-4d17-a03a-95a148fa6e64,913fad2e-c7cd-46f7-bef9-e16baba06f09,input,317,1074,1.0,1.0,True,True,True,None,True,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAA...,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAA...,mat-radio-input cdk-visually-hidden,None,radio,None,align-after-radio-button-input,angular
553,23ef1a88-6d8f-4746-ba3d-76a94bac243d,6ae76eff-6cc9-453e-9288-079b39a94c7a,input,413,1676,1.0,1.0,True,True,True,None,True,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAA...,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAA...,mat-radio-input cdk-visually-hidden,None,radio,None,slide-toggles-set_accent_color_radio-input,angular
757,ed813d00-cf99-46c2-8c9e-ac8a030a01cc,a3c4e0d4-0315-4195-8139-445235039836,option,17,0,0.0,0.0,True,True,True,Volvo,False,None,None,,None,None,None,,angular
869,f34a4eeb-b250-4793-a682-957af0a92f9c,faa5c8ab-19ff-45fc-9f0f-3244b7527dca,option,17,0,0.0,0.0,True,True,True,,False,None,None,,None,None,None,,angular
926,c644e173-922f-43f8-9c0a-8df6d0266c2d,7e8c4ea2-a3f7-4d6d-bd67-612d43c061e6,option,17,0,0.0,0.0,True,False,True,,False,None,None,,None,None,None,,angular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,04301d83-769d-4b93-b0f0-4f7adcd5cdd1,98f3a233-0b09-4066-8322-15ce7cd76343,option,17,0,0.0,0.0,True,True,True,Manager,False,None,None,,None,None,None,,user-table
246,6b889054-b531-4061-b0a0-b7b9313c8116,5b11bb91-6150-4886-b4fe-bc550a410e04,input,528,508,13.0,13.0,True,True,True,None,True,iVBORw0KGgoAAAANSUhEUgAAAA0AAAANCAYAAABy6+R8AA...,iVBORw0KGgoAAAANSUhEUgAAAA0AAAANCAYAAABy6+R8AA...,,None,checkbox,None,vlad,user-table
253,9d1b0573-ff8d-4640-ba31-7b1d1147a9b7,32e07905-35da-4f40-8b1b-42afefcca4b1,option,17,0,0.0,0.0,True,True,True,User,False,None,None,,None,None,None,,user-table
270,0bed1267-e8ce-4b3c-a6ed-43e4e20b0100,75e6bbf3-192e-431d-bc28-1b4fc7b09e3c,option,17,0,0.0,0.0,True,True,True,User,False,None,None,,None,None,None,,user-table


In [12]:
df[~df.attr_id.isnull()].size, 

331220

In [13]:
df.size

342300

In [15]:
df.groupby('attr_role').size()

attr_role
alert             3
button           23
columnheader     72
combobox          9
grid             18
gridcell        562
group            17
img              71
list              1
listbox          26
listitem          3
menu             39
navigation       19
option           12
progressbar      11
radiogroup       11
region            5
row             166
rowgroup         48
separator         3
slider            5
status            4
switch           12
tab              29
tablist          10
tabpanel         29
tree              5
treeitem          7
dtype: int64

In [16]:
df.groupby('attr_type').size()

attr_type
                   3046
button              110
checkbox            137
color                 4
date                  2
datetime-local        2
email                10
fieldset              3
file                  9
month                 2
number               19
password             28
radio                90
range                16
reset                 3
search               10
select-multiple       7
select-one           36
submit              195
tel                   1
text                188
text/css              3
text/javascript      48
textarea             15
time                  2
week                  2
dtype: int64

In [25]:
pd.DataFrame(df.groupby('attr_class').size()).sort_values(by=0, ascending=False)

,0
attr_class,
,7310
doc-space line,204
mat-button-wrapper,152
mat-button-focus-overlay,152
example-h2,148
...,...
mat-form-field-infix ng-tns-c95-80,1
mat-form-field-infix ng-tns-c95-81,1
mat-form-field-infix ng-tns-c95-82,1


In [26]:
pd.DataFrame(df.groupby('attr_onclick').size()).sort_values(by=0, ascending=False)

,0
attr_onclick,
"window.open('search.html','_self')",17
alert('Search Button Clicked!');,4
alert('Search');,3
alert('Red button');,3
alert('Grey button');,2
confirm('Suspend button');,2
alert('JDI Logo');,2
alert('JDI Title');,2
prompt('Ghost button');,2


In [27]:
pd.DataFrame(df.groupby('attr_type').size()).sort_values(by=0, ascending=False)

,0
attr_type,
,3046
submit,195
text,188
checkbox,137
button,110
radio,90
text/javascript,48
select-one,36
password,28
